In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Load and preprocess the data
data2 = pd.read_csv("C:/Users/Youness Azimzade/Dropbox/0Survival/Survival-XML-TME-BC/Data/MBRC.csv")
data = data2.copy()
data = data.drop(['Tumor.Stage'], axis=1)   # Remove unwanted column
data = data.dropna(axis=0)                  # Drop rows with missing values

# Encode categorical variables
her2 = {'Positive': 1, 'Negative': 0}
data.HER2 = data.HER2.map(her2)

cellul = {'Low': 1, 'Moderate': 2, 'High': 3}
data.Cellularity = data.Cellularity.map(cellul)

pgr = {'Positive': 1, 'Negative': 0}
data.PgR = data.PgR.map(pgr)

er = {'Positive': 1, 'Negative': 0}
data.ER = data.ER.map(er)

# One-hot encode the PAM50 column
categorical_cols = ['PAM50']
data = pd.get_dummies(data, columns=categorical_cols)

# Additional configuration and imports
from sklearn import set_config
from sklearn.model_selection import train_test_split, GridSearchCV
from sksurv.metrics import concordance_index_censored
set_config(display="text")
sns.set_style("whitegrid")

# Prepare features (X) and target (y)
X = data.drop(['Mixture', 'RFSurvival', 'RFStatus'], axis=1)
d3 = X.copy()  # Retain column names after scaling
y = data[['RFStatus', 'RFSurvival']]

# Standardize the feature set
from sklearn import preprocessing
X = pd.DataFrame(preprocessing.StandardScaler().fit_transform(X), columns=d3.columns)

print("X shape:", X.shape)
print("y shape:", y.shape)

# Calculate and print percentage of censored records
n_censored = y.shape[0] - y["RFStatus"].sum()
print("%.1f%% of records are censored" % (n_censored / y.shape[0] * 100))

# Convert y into a structured array required for survival analysis
aux = [(e1, e2) for e1, e2 in y.to_numpy()]
ny = np.array(aux, dtype=[('RFStatus', '?'), ('RFSurvival', '<f8')])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, ny, test_size=0.33333, random_state=10)

# Define a custom scoring function using concordance index
def cindex_scorer(estimator, X, y):
    predictions = estimator.predict(X)
    return concordance_index_censored(y['RFStatus'], y['RFSurvival'], predictions)[0]

# Tune parameters for the Cox Proportional Hazards model
from sksurv.linear_model import CoxPHSurvivalAnalysis

# Create a dynamic parameter grid for 'alpha' using a list comprehension
alpha_values = [1.5**(j) for j in range(-10, 10)]
param_grid = {"alpha": alpha_values}

# Set up GridSearchCV using the custom scoring function
grid = GridSearchCV(CoxPHSurvivalAnalysis(), param_grid, cv=5, scoring=cindex_scorer)
grid.fit(X_train, y_train)

print("Best parameters for Cox PH:", grid.best_params_)
print("Best cross-validated concordance index:", grid.best_score_)

# Fit the final model with the best parameters
cph_model = grid.best_estimator_
cph_model.fit(X_train, y_train)

# Evaluate the final model using the concordance index on the test set
cindex_cph = cph_model.score(X_test, y_test)
print("Test-set C-index for Cox Proportional Hazards model:", round(cindex_cph, 3))


X shape: (1764, 41)
y shape: (1764, 2)
59.2% of records are censored
Best parameters for Cox PH: {'alpha': 38.443359375}
Best cross-validated concordance index: 0.6537163668322161
Test-set C-index for Cox Proportional Hazards model: 0.643
